# Imports

In [1]:
import numpy as np
import pandas as pd

from functools import reduce


import plotly.express as px
import plotly.graph_objects as go



import warnings
warnings.filterwarnings('ignore')

In [73]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


In [74]:
dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung']
    
    
    

## ToDo

- % Zuwachs: Either since 2015 or Benutzerdefiniert
- Waterfall graphs for %, stacked barchart for Gesamtvolum
- Top/Bottom 10
- DF to the bottom

Filter Steps:
 1. Only BMZ
 2. 100% of full, 50% of Partial Financing

# World

### Basic Processing of Data


In [209]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



In [210]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


globe_df = globe_df.loc[[142]]



### Prepare data for stacked barchart


In [212]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000



melted_df.to_csv("../upload_data/globe_df.csv")

In [213]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,...,non_clim_amount_2013,non_clim_amount_2014,non_clim_amount_2015,non_clim_amount_2016,non_clim_amount_2017,non_clim_amount_2018,non_clim_amount_2019,non_clim_amount_2020,non_clim_amount_2021,non_clim_amount_2022
142,5627.252576,845.563368,6872.547503,1036.792467,5880.075415,995.181229,7376.387963,1352.222299,7670.370692,1457.414366,...,4781.689208,5835.755036,4884.894186,6024.165664,6212.956326,6837.718702,7118.226759,8623.937708,8626.519826,9238.493688


### Plot stacked bar

In [174]:
read_df = pd.read_csv('../upload_data/globe_df')


fig = px.bar(read_df, x='Year', y='Amount', color='Type',
            title='Global Financing Totals',
            labels={'Amount': 'Financing Amount (Euros)', 'Year': 'Year'},
            category_orders={'Type': ['Other Funds','Climate Finance']},
            color_discrete_map={'Other Funds': 'orange', 'Climate Finance': 'green'})# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

### Percentage

In [ ]:
years = [col.split('_')[-1] for col in globe_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    globe_df[clim_rel_percent_col] = (globe_df[clim_rel_amount_col] / globe_df[amount_col]) * 100
    
    

In [ ]:
selected_columns = [col for col in globe_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = globe_df[selected_columns]

In [ ]:

# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')

In [ ]:
melted_df

In [ ]:
melted_df['Change'] = melted_df['Percentage'].diff()

In [ ]:
melted_df.to_csv('../upload_data/globe_waterfall.csv')

In [19]:
globe_waterfall_df = pd.read_csv('../upload_data/globe_waterfall.csv')


In [30]:
globe_waterfall_df = pd.read_csv('../upload_data/globe_waterfall.csv')

initial_value = globe_waterfall_df['Percentage'].iloc[0] - globe_waterfall_df['Change'].iloc[1] # Subtract the first actual change to get the starting point


fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["absolute"] + ["relative"] * (len(globe_waterfall_df) - 1), # The first measure is absolute, others are relative
    x = globe_waterfall_df['Year'].astype(str),
    textposition = "outside",
    text = globe_waterfall_df['Change'].round(2).astype(str),
    y = [initial_value] + globe_waterfall_df['Change'].tolist()[1:], # The initial value plus the changes
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = {
            'text': "Yearly Percentage Change in Climate Finance",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis = {"type":"category"},
        yaxis = {"title":"Percentage"},
)
fig.show()

### Nice DF

In [ ]:
pivot_df = df.pivot_table(index='Year', columns='Type', values='Amount', aggfunc='sum').reset_index()


In [62]:
pivot_df = read_df[['Amount', 'Year', 'Type']].pivot_table(
    index='Year', 
    columns='Type', 
    values='Amount', aggfunc='sum').reset_index()





In [67]:
full_globe_df = pivot_df.merge(globe_waterfall_df, left_on="Year", right_on="Year").set_index('Year').drop(columns='Unnamed: 0')

In [69]:
full_globe_df.to_csv('../upload_data/globe_df_to_show.csv')

In [70]:
full_globe_df = pd.read_csv('../upload_data/globe_df_to_show.csv')

In [71]:
full_globe_df

,Year,Climate Finance,Other Funds,Percentage,Change
0,2013,8.455634e+08,4.781689e+09,15.026220,NaN
1,2014,1.036792e+09,5.835755e+09,15.085999,0.059779
2,2015,9.951812e+08,4.884894e+09,16.924634,1.838634
3,2016,1.352222e+09,6.024166e+09,18.331768,1.407134
4,2017,1.457414e+09,6.212956e+09,19.000573,0.668805
5,2018,1.659673e+09,6.837719e+09,19.531556,0.530983
6,2019,2.007335e+09,7.118227e+09,21.996836,2.465280
7,2020,2.195044e+09,8.623938e+09,20.288824,-1.708012
8,2021,2.490046e+09,8.626520e+09,22.399419,2.110595
9,2022,2.830265e+09,9.238494e+09,23.451170,1.051751


# Country Sum

In [164]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)


df_merged.to_csv("../upload_data/all_country_df.csv")


In [166]:

all_country_df = pd.read_csv("../upload_data/all_country_df.csv")

countries = all_country_df['Recipient Name'].unique()

selected_countries = ['India', 'Brazil', 'Namibia', 'Ukraine', 'Tunisia', 'Mexico']

all_country_df = all_country_df[all_country_df['Recipient Name'].isin(selected_countries)]


selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = all_country_df[selected_columns]

sums = filtered_df.sum()

sum_df = pd.DataFrame(sums).transpose()


sum_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
0,292.067278,123.703755,275.849281,93.278086,306.3415,89.92309,383.758265,131.627719,436.842413,173.973923,396.103788,144.384273,357.591627,141.713301,337.568699,94.110152,503.857364,141.409139,816.576266,124.62496


In [152]:
years = range(2013, 2023)
for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    sum_df[non_clim_col] = sum_df[amount_col] - sum_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = sum_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000




fig = px.bar(melted_df, x='Year', y='Amount', color='Type',
            title='Global Financing Totals',
            labels={'Amount': 'Financing Amount (Euros)', 'Year': 'Year'},
            category_orders={'Type': ['Other Funds','Climate Finance']},
            color_discrete_map={'Other Funds': 'orange', 'Climate Finance': 'green'})# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

In [180]:
sum_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
0,292.067278,123.703755,275.849281,93.278086,306.3415,89.92309,383.758265,131.627719,436.842413,173.973923,396.103788,144.384273,357.591627,141.713301,337.568699,94.110152,503.857364,141.409139,816.576266,124.62496


In [181]:
years = [col.split('_')[-1] for col in sum_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    sum_df[clim_rel_percent_col] = (sum_df[clim_rel_amount_col] / sum_df[amount_col]) * 100
    
    

In [182]:
sum_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,...,clim_rel_percent_2013,clim_rel_percent_2014,clim_rel_percent_2015,clim_rel_percent_2016,clim_rel_percent_2017,clim_rel_percent_2018,clim_rel_percent_2019,clim_rel_percent_2020,clim_rel_percent_2021,clim_rel_percent_2022
0,292.067278,123.703755,275.849281,93.278086,306.3415,89.92309,383.758265,131.627719,436.842413,173.973923,...,42.354541,33.814874,29.353871,34.299644,39.825328,36.451121,39.629927,27.878815,28.065312,15.261889


In [183]:
selected_columns = [col for col in sum_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = sum_df[selected_columns]

In [188]:

# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')
melted_df['Change'] = melted_df['Percentage'].diff()

In [191]:
initial_value = melted_df['Percentage'].iloc[0] - melted_df['Change'].iloc[1] # Subtract the first actual change to get the starting point


fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["absolute"] + ["relative"] * (len(melted_df) - 1), # The first measure is absolute, others are relative
    x = melted_df['Year'].astype(str),
    textposition = "outside",
    text = melted_df['Change'].round(2).astype(str),
    y = [initial_value] + melted_df['Change'].tolist()[1:], # The initial value plus the changes
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = {
            'text': "Yearly Percentage Change in Climate Finance for Selected Countries",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis = {"type":"category"},
        yaxis = {"title":"Percentage"},
)
fig.show()

# Country Compare

# Country Breakdown

In [193]:
use_df = dictionary_df['2022'][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [194]:

use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

In [195]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

In [196]:
use_df['clim_rel'] = use_df.apply(determine_value, axis=1)


In [197]:
use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']

In [198]:
grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]

In [199]:
grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']

In [200]:
grouped_df

,Amount,clim_rel_amount,clim_rel_percent
Recipient Name,,,
Afghanistan,201.939382,12.035070,0.059597
"Africa, regional",627.644704,138.891343,0.221290
Albania,25.328362,8.592429,0.339241
Algeria,16.273651,7.171532,0.440684
"America, regional",107.665968,50.903984,0.472795
...,...,...,...
West Bank and Gaza Strip,150.189918,17.381477,0.115730
"Western Africa, regional",37.365181,12.119308,0.324348
Yemen,240.614682,4.985232,0.020719


In [204]:
processed_grouped_dfs = []

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

for key in dictionary_df.keys():
        
    use_df = dictionary_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']].reset_index()
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_processed_df = grouped_df.rename(columns={'clim_rel_percent': key})

    processed_grouped_dfs.append(grouped_processed_df[['Recipient Name',key]])

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


# df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long.to_csv('../upload_data/newmethod_country_specific_df.csv')


In [ ]:
df_long

In [ ]:
processed_grouped_dfs = []

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']

    use_df.loc[use_df['Climate Relevance'] > 1, 'Climate Relevance'] = 1

    first_group_df = use_df.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']].reset_index()

    df_grouped = first_group_df.groupby('Recipient Name').apply(
        lambda x: (x[x['Climate Relevance'] == 1]['Amount'].sum()\
                   / x['Amount'].sum()) * 100).reset_index(name='Climate Relevance %')

    grouped_processed_df = df_grouped.rename(columns={'Climate Relevance %': key})

    processed_grouped_dfs.append(grouped_processed_df)

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long

In [ ]:
df_long = pd.read_csv('../upload_data/country_specific_df.csv')[['Recipient Name', 'Year', 'Value']]

In [ ]:
df_long

In [ ]:
countries = df_long['Recipient Name'].unique()
# countries

In [214]:
df_long.to_csv('../upload_data/country_specific_df.csv')

In [226]:
selected_df = df_long[df_long['Recipient Name'].isin(['India', 'Brazil', 'Ukraine', 'Namibia', 'South Africa'])]

In [227]:
selected_df

,Recipient Name,Year,Value
19,Brazil,2013,0.713478
58,India,2013,0.293126
84,Namibia,2013,0.131611
106,South Africa,2013,0.293300
123,Ukraine,2013,0.098188
161,Brazil,2014,0.578425
200,India,2014,0.346076
226,Namibia,2014,0.152328
248,South Africa,2014,0.246258
265,Ukraine,2014,0.177336


In [236]:
from_year, to_year = str(2016), str(2020)

In [235]:
selected_df.loc[1401]['Year']

'2022'

In [273]:
from_values = selected_df[selected_df['Year'] == from_year]\
        .sort_values(by = 'Recipient Name')\
        .reset_index()['Value']

to_values = selected_df[selected_df['Year'] == to_year]\
        .sort_values(by = 'Recipient Name')\
        .reset_index()['Value']



In [274]:
index_countries = list(selected_df[selected_df['Year'] == to_year].sort_values(by = 'Recipient Name')['Recipient Name'])


values_countries = to_values - from_values

values_countries.index = index_countries

ranked_df = pd.DataFrame(values_countries).rename(columns={'Value': '% Change in Period'})\
        .sort_values('% Change in Period', ascending = False)




In [285]:
ranked_df

,% Change in Period
Namibia,0.081315
Ukraine,0.048444
Brazil,-0.004073
India,-0.139166
South Africa,-0.302048


In [308]:
country_compare_df = pd.read_csv('../upload_data/country_specific_df.csv')

selected_countries = ['India', 'Brazil', 'Namibia', 'Ukraine', 'Tunisia', 'Mexico']

from_year, to_year = 2016, 2020

country_compare_df = country_compare_df[country_compare_df['Recipient Name'].isin(selected_countries)]

country_compare_df = country_compare_df[country_compare_df['Year'].between(from_year,to_year)]

from_values = country_compare_df[country_compare_df['Year'] == from_year]\
    .sort_values(by = 'Recipient Name')\
    .reset_index()['Value']

to_values = country_compare_df[country_compare_df['Year'] == to_year]\
        .sort_values(by = 'Recipient Name')\
        .reset_index()['Value']

index_countries = list(country_compare_df[country_compare_df['Year'] == to_year].sort_values(by = 'Recipient Name')['Recipient Name'])

values_countries = (to_values - from_values)*100

values_countries.index = index_countries

ranked_df = pd.DataFrame(values_countries).rename(columns={'Value': '% Change in Period'})\
        .sort_values('% Change in Period', ascending = False)


ranked_df

,% Change in Period
Namibia,8.131535
Ukraine,4.844352
Brazil,-0.407336
Mexico,-4.211752
India,-13.916630
Tunisia,-15.855018


In [312]:
country_compare_df = pd.read_csv('../upload_data/country_specific_df.csv')

from_year, to_year = 2018, 2022
country_compare_df = country_compare_df[country_compare_df['Year'].between(from_year,to_year)]

from_values = country_compare_df[country_compare_df['Year'] == from_year]\
    .sort_values(by = 'Recipient Name')\
    .reset_index()['Value']

to_values = country_compare_df[country_compare_df['Year'] == to_year]\
        .sort_values(by = 'Recipient Name')\
        .reset_index()['Value']

index_countries = list(country_compare_df[country_compare_df['Year'] == to_year].sort_values(by = 'Recipient Name')['Recipient Name'])

values_countries = to_values - from_values

values_countries.index = index_countries

ranked_df = pd.DataFrame(values_countries).rename(columns={'Value': '% Change in Period'})\
        .sort_values('% Change in Period', ascending = False)


ranked_df.dropna().head(10)

,% Change in Period
Malaysia,0.500000
C(te d'Ivoire,0.470015
Cuba,0.429642
Egypt,0.398306
Chad,0.320489
Georgia,0.299778
Peru,0.279918
Pakistan,0.256918
"South of Sahara, regional",0.252498
Armenia,0.244363


In [306]:
ranked_df.dropna().tail(10)

,% Change in Period
Algeria,-0.176109
Georgia,-0.190607
Panama,-0.229748
Liberia,-0.230490
Senegal,-0.244335
Paraguay,-0.258298
Democratic People's Republic of Korea,-0.262539
South Africa,-0.302048
Congo,-0.347662
Venezuela,-0.543966


In [207]:
fig = px.line(selected_df, 
              x='Year', 
              y='Value', 
              color='Recipient Name',
              title='Value by Year for Selected Country',
              labels={'Value': 'Climate Relevant Financing (%)', 'Year': 'Year'},
              markers=True)

fig.show()

# Globe Graph

In [ ]:
def df_compiler(dictionary_of_df):
    
    processed_grouped_dfs = []
    
    for key in dictionary_of_df.keys():
        
        use_df = dictionary_of_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
        
        use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
        use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
        
        use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']
        
        use_df.loc[use_df['Climate Relevance'] > 2, 'Climate Relevance'] = 2
        
        use_df['Amount'] = use_df['Amount'] * 1_000_000
        
        grouped_processed_df = use_df.groupby(['Climate Relevance']).sum()[['Amount']]
        
        grouped_processed_df = grouped_processed_df.rename(columns={'Amount': key})
        
        processed_grouped_dfs.append(grouped_processed_df)
        
    merged_df = pd.concat(processed_grouped_dfs, axis=1)
    
    sum_row = merged_df.sum()
    sum_df = pd.DataFrame([sum_row], index=['Total Financing'])
    
    merged_df = merged_df.append(sum_df)

    merged_df.index = ['No Climate Relevance','Partial Climate Relevance',
                       'High Climate Relevance', 'Total Financing']

    return merged_df


merged_df = df_compiler(df_dictionary)

merged_df.to_csv('../upload_data/global_df.csv')

merged_df

In [ ]:
globe_df = pd.read_csv('../upload_data/global_df.csv')
globe_df

In [ ]:

df_long = merged_df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']



fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Global BMZ Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:
df_long = df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']

# Filter out the 'Total Financing' row to avoid double counting in the stacked bar
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']

# Create the stacked bar chart
fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Total Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:


testing_2022 = df_2022[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:
testing_2022['Climate Adaptation'] = testing_2022['Climate Adaptation'].fillna(0)
testing_2022['Climate Mitigation'] = testing_2022['Climate Mitigation'].fillna(0)

In [ ]:
testing_2022['Climate Relevance'] = testing_2022['Climate Adaptation'] + testing_2022['Climate Mitigation']

In [ ]:
testing_2022.loc[testing_2022['Climate Relevance'] > 2, 'Climate Relevance'] = 2


In [ ]:
testing_2022.groupby(['Climate Relevance']).sum()[['Amount']].rename(columns={'Amount': '2022'})

In [ ]:
grouped_test = testing_2022.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']]